In [56]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# plt.style.use('ggplot')
font = {'weight': 'bold','size':   16}
plt.rc('font', **font)
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geopy.distance as geo
import re

In [57]:
rtd_df = pd.read_csv('../data/rtd_data.csv')

In [58]:
rtd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453089 entries, 0 to 453088
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   entity_id              453088 non-null  object 
 1   trip_id                453088 non-null  float64
 2   schedule_relationship  453088 non-null  float64
 3   route_id               453088 non-null  object 
 4   direction_id           453088 non-null  float64
 5   vehicle_lat            453088 non-null  float64
 6   vehicle_lng            453088 non-null  float64
 7   bearing                453088 non-null  float64
 8   current_status         453088 non-null  float64
 9   timestamp              453088 non-null  float64
 10  stop_id                453088 non-null  float64
 11  vehicle_id             453088 non-null  object 
 12  vehicle_label          453088 non-null  float64
dtypes: float64(10), object(3)
memory usage: 44.9+ MB


In [59]:
def join_txt_file(df, txt_file, join_type, join_columns):
        '''
        Joins a txt column to a pandas DataFrame by the join_column(s) using the type of join passed ('left', 'outer')
        
        Args:
            txt_file (str): Filepath to a text file that is separated by ','

            join_type (str): The type of join to use (Left, Outer, Inner, Full)

            join_columns (list): The columns to perform the join with
        '''
        return pd.merge(df, pd.read_csv(txt_file, delimiter=','), how=join_type, on=join_columns, suffixes=('', '_joined')) 


In [60]:
def convert_timezone_local(df, colname_list, current_timezone, local_timezone): 
        '''
        Converts the columns in colname_list to local timezone

        Args:
            colname_list (list): list of column names (strings) to convert in the RTD_df.df class. 
                                Values should be int for conversion
            current_timezone (str): string of timezone that the column name currently is in
            local_timezone (str): string of the timezone that the column name should be converted to.
        '''
        for column in colname_list:
            df[column] = pd.to_datetime(df[column], unit='s').dt.tz_localize(current_timezone).dt.tz_convert(local_timezone)
        
        return df

In [61]:
def parse_codes(df, colname, code_dict):
        '''
        Converts coded integers to their real-world values given in the code_dict

        Args:
            colname (str): column name of dataframe that contains the coded values to be converted

            code_dict (dict): dictionary of code conversion where the keys are the code integers and values 
            are their corresponding real-world values.
        '''
        df[colname] = df[colname].replace(to_replace = code_dict)

        return df

In [62]:
def calculate_distance(df, point_1_lat, point_1_lng, point_2_lat, point_2_lng, new_column):
        '''
        Calculates the distance between two points in meters given their lat/lng

        Args: 
            point_1_lat (str): The name of the column with the first points latitudes

            point_1_lng (str): The name of the column with the first points longitudes

            point_2_lat (str): The name of the column with the second points latitudes

            point_2_lng (str): The name of the column with the second points longitudes

            new_column (str): The name of the new column created from the lat/lng points
        '''
        point_1 = list(zip(df[point_1_lat], df[point_1_lng]))
        point_2 = list(zip(df[point_2_lat], df[point_2_lng]))
        
        df[new_column] = [round(geo.distance(point_1, point_2).m,2) if (~pd.isnull(point_2[0])) & (point_1[0] > 0) else np.nan for point_1, point_2 in zip(point_1, point_2)]
        
        return df

In [63]:
def calculate_time(df, time_1, time_2, new_column):
        '''
        Calculates the time in minutes between a timestamp and a string in the format %H:%M:%S

        Args:
            time_1 (str): The name of the column with timestamp values to use for the first time

            time_2 (str): The name of the column with string values in the format %H:%M:%S to use for the second time

            new_column (str): The name of the new column created from the difference in times
        '''
        time_diff = pd.to_datetime(df[time_1]).dt.tz_localize(None) - df[time_2].apply(lambda x: pd.Timestamp(x))
        df[new_column] = round(time_diff.apply(lambda x: x.total_seconds()/60), 2)
        
        return df

In [64]:
rtd_df = join_txt_file(rtd_df, '~/Documents/dsi/repos/capstone_1/data/google_transit/routes.txt', 'left', ['route_id'])
rtd_df = join_txt_file(rtd_df, '~/Documents/dsi/repos/capstone_1/data/google_transit/trips.txt', 'left', ['trip_id'])
rtd_df = join_txt_file(rtd_df, '~/Documents/dsi/repos/capstone_1/data/google_transit/stops.txt', 'left', ['stop_id'])
rtd_df = join_txt_file(rtd_df, '~/Documents/dsi/repos/capstone_1/data/google_transit/stop_times.txt', 'left', ['trip_id', 'stop_id'])

In [65]:
rtd_df = rtd_df[(~rtd_df.stop_name.isnull()) & (~rtd_df.departure_time.isnull())]

In [66]:
rtd_df = rtd_df.loc[:,['entity_id'
                  ,'trip_id'
                  ,'trip_headsign'
                  ,'route_id'
                  ,'route_long_name'
                  ,'route_short_name'
                  ,'route_type'
                  ,'route_desc'
                  ,'vehicle_lat'
                  ,'vehicle_lng'
                  ,'bearing'
                  ,'current_status'
                  ,'timestamp'
                  ,'stop_id'
                  ,'stop_name'
                  ,'stop_desc'
                  ,'stop_lat'
                  ,'stop_lon'
                  ,'arrival_time'
                  ,'departure_time'
                  ,'vehicle_id'
                  ,'vehicle_label']]

In [67]:
status_dict = {0: 'incoming_at'
                  ,1: 'stopped_at'
                  ,2: 'in_transit_to'}

route_dict = {0: 'light_rail'
             ,2: 'commuter_rail'
             ,3: 'bus'}

rtd_df = parse_codes(rtd_df, 'current_status', status_dict)
rtd_df = parse_codes(rtd_df, 'route_type', route_dict)
rtd_df = convert_timezone_local(rtd_df, ['timestamp'], 'UTC', 'US/Mountain')

In [68]:
rtd_df.rename({'timestamp': 'arrival_timestamp'
              ,'arrival_time': 'scheduled_arrival_time'
              ,'departure_time': 'scheduled_departure_time'
              ,'vehicle_lat': 'arrival_vehicle_lat'
              ,'vehicle_lng': 'arrival_vehicle_lng'
              ,'stop_lon': 'stop_lng'}, axis=1, inplace=True)

In [74]:
rtd_df = rtd_df.replace({'scheduled_arrival_time': {r'^24':'00'
                                                    ,r'^25':'00'}
                        ,'scheduled_departure_time': {r'^24':'00'
                                                      ,r'^25':'00'}
                        }, regex=True)
# rtd_df.scheduled_departure_time.apply(lambda x: x[0:2]).unique()

In [75]:
# Sort by trip, vehicle, and then timestamp to follow a vehicle from one point to the next in chronological order on their trip
sorted_rtd_df = rtd_df.sort_values(['trip_id', 'vehicle_id', 'arrival_timestamp'])
# Lag timestamp by -1 to get the departure time when a vehicle leaves a stop
sorted_rtd_df.loc[:,'departure_timestamp'] = sorted_rtd_df.arrival_timestamp.shift(-1)
sorted_rtd_df.loc[:,'departure_vehicle_lat'] = sorted_rtd_df.arrival_vehicle_lat.shift(-1)
sorted_rtd_df.loc[:,'departure_vehicle_lng'] = sorted_rtd_df.arrival_vehicle_lng.shift(-1)

In [76]:
sorted_rtd_df = calculate_time(sorted_rtd_df, 'arrival_timestamp', 'scheduled_arrival_time', 'minutes_to_arrival')
sorted_rtd_df = calculate_time(sorted_rtd_df, 'departure_timestamp', 'scheduled_departure_time', 'minutes_since_departure')
sorted_rtd_df = calculate_distance(sorted_rtd_df, 'arrival_vehicle_lat', 'arrival_vehicle_lng', 'stop_lat', 'stop_lng', 'meters_to_arrival')
sorted_rtd_df = calculate_distance(sorted_rtd_df, 'departure_vehicle_lat', 'departure_vehicle_lng', 'stop_lat', 'stop_lng', 'meters_since_departure')

In [88]:
# Use rank to determine when a vehicle switches from one stop to the next, indicating they have left the first stop and are in transit to the next.
sorted_rtd_df.loc[:, 'stop_rank'] = sorted_rtd_df.groupby(['trip_id', 'vehicle_id', 'stop_id']).arrival_timestamp.rank(axis=1, method='min', ascending=False)

sorted_rtd_df[sorted_rtd_df.trip_id == 113655878.0].loc[:, ['trip_id', 'vehicle_id', 'stop_id', 'arrival_timestamp', 'stop_rank']]

,trip_id,vehicle_id,stop_id,arrival_timestamp,stop_rank
24410,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23051.0,2021-02-16 06:53:03-07:00,5.0
24668,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23051.0,2021-02-16 06:53:03-07:00,5.0
29968,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23048.0,2021-02-16 07:14:03-07:00,1.0
30240,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:15:47-07:00,20.0
30511,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:15:47-07:00,20.0
30784,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:15:47-07:00,20.0
31055,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:18:39-07:00,1.0
31324,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:18:39-07:00,1.0
31592,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:18:39-07:00,1.0
31861,113655878.0,BB6C950F53250192E053DD4D1FAC7790,23049.0,2021-02-16 07:18:39-07:00,1.0


In [134]:
total_stops = rtd_df.groupby(['route_id', 'stop_name']).trip_id.nunique().reset_index()
total_stops.trip_id.sum()

148275

In [135]:
rtd_df.head(100)

,entity_id,trip_id,trip_headsign,route_id,route_long_name,route_short_name,route_type,route_desc,vehicle_lat,vehicle_lng,bearing,current_status,timestamp,stop_id,stop_name,stop_desc,stop_lat,stop_lon,arrival_time,departure_time,vehicle_id,vehicle_label,meters_from_stop,minutes_from_stop
2,1613458791_1541,113632619.0,8th/Coffman,BOLT,Boulder / Longmont,BOLT,bus,This Route Travels Northbound & Southbound,40.107483,-105.169868,41.0,in_transit_to,2021-02-15 23:58:56-07:00,19973.0,Hwy 119 & Airport Rd,Vehicles Travelling Northeast,40.121142,-105.151456,23:58:20,23:58:20,1541,1541.0,2182.70,-1439.40
3,1613458791_3615,113624102.0,Downtown Boulder via E-470,AB1,Boulder / Denver Airport,AB1,bus,This Route Travels Eastbound & Westbound,39.964565,-105.182549,297.0,in_transit_to,2021-02-15 23:59:20-07:00,33855.0,US 36 & Table Mesa Station Gate B,Vehicles Travelling East,39.986706,-105.232717,00:15:00,00:15:00,3615,3615.0,4940.65,-15.67
5,1613458791_3668,113624976.0,Union Station All-Station,FF1,Flatiron Flyer,FF1,bus,This Route Travels Eastbound & Westbound,39.887917,-105.073082,152.0,in_transit_to,2021-02-15 23:59:10-07:00,34660.0,US 36 & Sheridan Station Gate C,Vehicles Travelling East,39.857430,-105.054121,00:03:00,00:03:00,3668,3668.0,3753.60,-3.83
6,1613458791_3712,113625019.0,Downtown Boulder All-Station,FF1,Flatiron Flyer,FF1,bus,This Route Travels Eastbound & Westbound,39.986099,-105.250252,332.0,stopped_at,2021-02-15 23:59:15-07:00,12604.0,S Broadway & Table Mesa Dr,Vehicles Travelling Northwest,39.986040,-105.250119,00:04:00,00:04:00,3712,3712.0,13.11,-4.75
7,1613458791_5106,113622933.0,Commerce City & 72nd Ave Stn via 72nd Ave,88,Central Park/ 72nd Ave Station,88,bus,This Route Travels Northbound & Southbound,39.767117,-104.891068,106.0,in_transit_to,2021-02-15 23:59:17-07:00,34975.0,Central Park Blvd & 46th Pl,Vehicles Travelling North,39.781356,-104.882635,00:00:11,00:00:11,5106,5106.0,1738.25,-0.90
8,1613458791_5109,113619309.0,Downtown,38,West 38th Avenue,38,bus,This Route Travels Eastbound & Westbound,39.747784,-104.990669,40.0,stopped_at,2021-02-15 23:58:56-07:00,17761.0,Stout St & 18th St,Vehicles Travelling Northeast,39.747524,-104.990715,23:59:00,23:59:00,5109,5109.0,29.10,-1440.07
9,1613458791_5114,113619334.0,Ward Rd & I-70 Park-n-Ride,38,West 38th Avenue,38,bus,This Route Travels Eastbound & Westbound,39.780533,-105.138618,188.0,in_transit_to,2021-02-15 23:59:06-07:00,35236.0,Ward Rd PnR (Ar),Vehicles Travelling South,39.781251,-105.138657,23:55:00,23:55:00,5114,5114.0,79.81,-1435.90
10,1613458791_5122,113633891.0,106th&Melody,31,Federal Blvd,31,bus,This Route Travels Northbound & Southbound,39.737701,-105.025002,10.0,stopped_at,2021-02-15 23:59:19-07:00,13703.0,Decatur / Federal Station Gate E,Vehicles Travelling North,39.737734,-105.024986,23:59:00,00:00:00,5122,5122.0,3.85,-0.68
11,1613458791_5125,113622084.0,Arapahoe Community College,66,Arapahoe Road,66,bus,This Route Travels Eastbound & Westbound,39.595135,-104.910950,270.0,in_transit_to,2021-02-15 23:59:07-07:00,12281.0,Arapahoe Rd & S Krameria Way,Vehicles Travelling West,39.595134,-104.916031,23:58:35,23:58:35,5125,5125.0,436.46,-1439.47
12,1613458791_5126,113633994.0,Evans,31,Federal Blvd,31,bus,This Route Travels Northbound & Southbound,39.758232,-105.025314,180.0,in_transit_to,2021-02-15 23:59:20-07:00,13714.0,Federal Blvd & W 29th Ave,Vehicles Travelling South,39.758669,-105.025336,23:58:09,23:58:09,5126,5126.0,48.54,-1438.82


In [136]:
time_play = rtd_df.loc[:,['timestamp', 'arrival_time', 'departure_time']]
day_date = time_play.timestamp.dt.to_period('D').astype(str)
rtd_df['arrival_timestamp'] = pd.to_datetime(day_date + ' ' + time_play.arrival_time)
rtd_df['departure_timestamp'] = pd.to_datetime(day_date + ' ' + time_play.departure_time)
time_diff = rtd_df['timestamp'].dt.tz_localize(None) - rtd_df['departure_timestamp'].dt.tz_localize(None)
rtd_df['time_to_departure'] = round(time_diff.apply(lambda x: x.total_seconds()/60), 2)
rtd_df.iloc[0:10,:]

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1104: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,entity_id,trip_id,trip_headsign,route_id,route_long_name,route_short_name,route_type,route_desc,vehicle_lat,vehicle_lng,bearing,current_status,timestamp,stop_id,stop_name,stop_desc,stop_lat,stop_lon,arrival_time,departure_time,vehicle_id,vehicle_label,meters_from_stop,minutes_from_stop,arrival_timestamp,departure_timestamp,time_to_departure
2,1613458791_1541,113632619.0,8th/Coffman,BOLT,Boulder / Longmont,BOLT,bus,This Route Travels Northbound & Southbound,40.107483,-105.169868,41.0,in_transit_to,2021-02-15 23:58:56-07:00,19973.0,Hwy 119 & Airport Rd,Vehicles Travelling Northeast,40.121142,-105.151456,23:58:20,23:58:20,1541,1541.0,2182.70,-1439.40,2021-02-15 23:58:20,2021-02-15 23:58:20,0.60
3,1613458791_3615,113624102.0,Downtown Boulder via E-470,AB1,Boulder / Denver Airport,AB1,bus,This Route Travels Eastbound & Westbound,39.964565,-105.182549,297.0,in_transit_to,2021-02-15 23:59:20-07:00,33855.0,US 36 & Table Mesa Station Gate B,Vehicles Travelling East,39.986706,-105.232717,00:15:00,00:15:00,3615,3615.0,4940.65,-15.67,2021-02-15 00:15:00,2021-02-15 00:15:00,1424.33
5,1613458791_3668,113624976.0,Union Station All-Station,FF1,Flatiron Flyer,FF1,bus,This Route Travels Eastbound & Westbound,39.887917,-105.073082,152.0,in_transit_to,2021-02-15 23:59:10-07:00,34660.0,US 36 & Sheridan Station Gate C,Vehicles Travelling East,39.857430,-105.054121,00:03:00,00:03:00,3668,3668.0,3753.60,-3.83,2021-02-15 00:03:00,2021-02-15 00:03:00,1436.17
6,1613458791_3712,113625019.0,Downtown Boulder All-Station,FF1,Flatiron Flyer,FF1,bus,This Route Travels Eastbound & Westbound,39.986099,-105.250252,332.0,stopped_at,2021-02-15 23:59:15-07:00,12604.0,S Broadway & Table Mesa Dr,Vehicles Travelling Northwest,39.986040,-105.250119,00:04:00,00:04:00,3712,3712.0,13.11,-4.75,2021-02-15 00:04:00,2021-02-15 00:04:00,1435.25
7,1613458791_5106,113622933.0,Commerce City & 72nd Ave Stn via 72nd Ave,88,Central Park/ 72nd Ave Station,88,bus,This Route Travels Northbound & Southbound,39.767117,-104.891068,106.0,in_transit_to,2021-02-15 23:59:17-07:00,34975.0,Central Park Blvd & 46th Pl,Vehicles Travelling North,39.781356,-104.882635,00:00:11,00:00:11,5106,5106.0,1738.25,-0.90,2021-02-15 00:00:11,2021-02-15 00:00:11,1439.10
8,1613458791_5109,113619309.0,Downtown,38,West 38th Avenue,38,bus,This Route Travels Eastbound & Westbound,39.747784,-104.990669,40.0,stopped_at,2021-02-15 23:58:56-07:00,17761.0,Stout St & 18th St,Vehicles Travelling Northeast,39.747524,-104.990715,23:59:00,23:59:00,5109,5109.0,29.10,-1440.07,2021-02-15 23:59:00,2021-02-15 23:59:00,-0.07
9,1613458791_5114,113619334.0,Ward Rd & I-70 Park-n-Ride,38,West 38th Avenue,38,bus,This Route Travels Eastbound & Westbound,39.780533,-105.138618,188.0,in_transit_to,2021-02-15 23:59:06-07:00,35236.0,Ward Rd PnR (Ar),Vehicles Travelling South,39.781251,-105.138657,23:55:00,23:55:00,5114,5114.0,79.81,-1435.90,2021-02-15 23:55:00,2021-02-15 23:55:00,4.10
10,1613458791_5122,113633891.0,106th&Melody,31,Federal Blvd,31,bus,This Route Travels Northbound & Southbound,39.737701,-105.025002,10.0,stopped_at,2021-02-15 23:59:19-07:00,13703.0,Decatur / Federal Station Gate E,Vehicles Travelling North,39.737734,-105.024986,23:59:00,00:00:00,5122,5122.0,3.85,-0.68,2021-02-15 23:59:00,2021-02-15 00:00:00,1439.32
11,1613458791_5125,113622084.0,Arapahoe Community College,66,Arapahoe Road,66,bus,This Route Travels Eastbound & Westbound,39.595135,-104.910950,270.0,in_transit_to,2021-02-15 23:59:07-07:00,12281.0,Arapahoe Rd & S Krameria Way,Vehicles Travelling West,39.595134,-104.916031,23:58:35,23:58:35,5125,5125.0,436.46,-1439.47,2021-02-15 23:58:35,2021-02-15 23:58:35,0.53
12,1613458791_5126,113633994.0,Evans,31,Federal Blvd,31,bus,This Route Travels Northbound & Southbound,39.758232,-105.025314,180.0,in_transit_to,2021-02-15 23:59:20-07:00,13714.0,Federal Blvd & W 29th Ave,Vehicles Travelling South,39.758669,-105.025336,23:58:09,23:58:09,5126,5126.0,48.54,-1438.82,2021-02-15 23:58:09,2021-02-15 23:58:09,1.18


In [148]:
# Sort by trip, vehicle, and then timestamp to follow a vehicle from one point to the next in chronological order on their trip
sorted_rtd_df = rtd_df.sort_values(['trip_id', 'vehicle_id', 'timestamp'])
# Lag timestamp by -1 to get the departure time when a vehicle leaves a stop
sorted_rtd_df.loc[:,'timestamp_lag'] = sorted_rtd_df.timestamp.shift(-1)
# Use rank to determine when a vehicle switches from one stop to the next, indicating they have left the first stop and are in transit to the next.
sorted_rtd_df.loc[:, 'stop_rank'] = sorted_rtd_df.groupby(['trip_id', 'vehicle_id', 'stop_id']).timestamp.rank(ascending=False)
# Only get the columns I want to use
final_rtd_df = sorted_rtd_df.iloc[:,[1,3,11,12,14,25,26,27,28]]
# Calculate the time between the scheduled departure time and the actual departure time
time_diff = final_rtd_df['timestamp_lag'].dt.tz_localize(None) - rtd_df['departure_timestamp'].dt.tz_localize(None)
final_rtd_df['departure_time'] = round(time_diff.apply(lambda x: x.total_seconds()/60), 2)
# Calculate whether or not the vehicle was on time.
'''
Bus and Light Rail are considered on-time if a departure from a location is no more than 1 minute early or 5 minutes after the scheduled departure time.
Commuter Rail is considered on-time if a departure from a location is no more than 0 minutes early or 5 minutes after the scheduled departure time.
'''
final_rtd_df.loc[:,'on_time'] = (final_rtd_df[(final_rtd_df.stop_rank == 1)].departure_time >= -1.0) & (final_rtd_df[(final_rtd_df.stop_rank == 1)].departure_time <= 5.0)
# Only pull in data where the on_time value isn't null (meaning the data is at a point in between stops)
on_time_data = final_rtd_df[~final_rtd_df.on_time.isnull()]
# Calculate the on_time arrival rate based on the total number of stops & how many of those stops were on_time == True
on_time_data.on_time.sum() / on_time_data.on_time.size

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0.8253732597551752

In [119]:
rtd_df[rtd_df.trip_id == 113607693.0].sort_values('timestamp')

,entity_id,trip_id,trip_headsign,route_id,route_long_name,route_short_name,route_type,route_desc,vehicle_lat,vehicle_lng,bearing,current_status,timestamp,stop_id,stop_name,stop_desc,stop_lat,stop_lon,arrival_time,departure_time,vehicle_id,vehicle_label,meters_from_stop,minutes_from_stop,arrival_timestamp,departure_timestamp,time_to_departure
167317,1613512371_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.547249,-104.997620,0.0,stopped_at,2021-02-16 14:52:23-07:00,26262.0,Highlands Ranch Town Center PnR Gate B,Vehicles Travelling Northwest,39.547191,-104.997556,14:52:00,1.40,9345,9345.0,8.44,0.38,2021-02-16 14:52:00,2021-02-16 14:52:00,0.38
167630,1613512431_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.547249,-104.997620,3.0,stopped_at,2021-02-16 14:53:24-07:00,26262.0,Highlands Ranch Town Center PnR Gate B,Vehicles Travelling Northwest,39.547191,-104.997556,14:52:00,2.28,9345,9345.0,8.44,1.40,2021-02-16 14:52:00,2021-02-16 14:52:00,1.40
167946,1613512491_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.547352,-104.997749,324.0,in_transit_to,2021-02-16 14:54:17-07:00,25602.0,Ridgeline Blvd & Highlands Ranch Pkwy,Vehicles Travelling North,39.549632,-104.996192,14:52:39,2.68,9345,9345.0,286.37,1.63,2021-02-16 14:52:39,2021-02-16 14:52:39,1.63
168269,1613512551_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.549900,-104.996269,345.0,in_transit_to,2021-02-16 14:55:20-07:00,26250.0,8900 Block Ridgeline Blvd,Vehicles Travelling North,39.553734,-104.996451,14:53:19,2.95,9345,9345.0,425.95,2.02,2021-02-16 14:53:19,2021-02-16 14:53:19,2.02
168593,1613512611_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.556351,-104.997086,267.0,in_transit_to,2021-02-16 14:56:16-07:00,26247.0,Ridgeline Blvd & Bluepoint Rd,Vehicles Travelling North,39.556684,-104.998190,14:54:00,3.13,9345,9345.0,101.88,2.27,2021-02-16 14:54:00,2021-02-16 14:54:00,2.27
168914,1613512671_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.558384,-104.995216,67.0,in_transit_to,2021-02-16 14:57:08-07:00,25676.0,Centennial Blvd & Plaza Dr,Vehicles Travelling Northwest,39.559222,-104.990265,14:56:05,2.38,9345,9345.0,435.58,1.05,2021-02-16 14:56:05,2021-02-16 14:56:05,1.05
169241,1613512731_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.561565,-104.989769,71.0,in_transit_to,2021-02-16 14:58:28-07:00,22454.0,Centennial Blvd & S Broadway,Vehicles Travelling Northeast,39.561986,-104.988490,14:57:21,1.90,9345,9345.0,119.42,1.12,2021-02-16 14:57:21,2021-02-16 14:57:21,1.12
169564,1613512791_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.562466,-104.987129,79.0,in_transit_to,2021-02-16 14:59:15-07:00,20378.0,S Broadway & County Line Rd,Vehicles Travelling North,39.565628,-104.988345,14:58:43,1.53,9345,9345.0,366.32,0.53,2021-02-16 14:58:43,2021-02-16 14:58:43,0.53
169883,1613512851_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.562466,-104.987129,80.0,in_transit_to,2021-02-16 15:00:15-07:00,20378.0,S Broadway & County Line Rd,Vehicles Travelling North,39.565628,-104.988345,14:58:43,2.55,9345,9345.0,366.32,1.53,2021-02-16 14:58:43,2021-02-16 14:58:43,1.53
170207,1613512911_9345,113607693.0,Union Station,0,South Broadway,0,bus,This Route Travels Northbound & Southbound,39.563667,-104.987167,326.0,in_transit_to,2021-02-16 15:01:16-07:00,20378.0,S Broadway & County Line Rd,Vehicles Travelling North,39.565628,-104.988345,14:58:43,3.73,9345,9345.0,240.06,2.55,2021-02-16 14:58:43,2021-02-16 14:58:43,2.55
